In [1]:
import psycopg2
import pandas as pd

sqluser = 'student'
dbname = 'mimic'
password = 'lipup'
host = 'luolabwkstn01.fsm.northwestern.edu'
port = '5432'
schema_name = 'mimiciii'

con = psycopg2.connect(dbname = dbname, user = sqluser, password = password, host = host, port = port)

# Labs

In [ ]:
query = """
select * from

(SELECT ie.subject_id, ie.hadm_id, ie.icustay_id, ceiling((extract( epoch from ie.outtime - le.charttime))/60/60/24) as day
  -- the day to outtime
  , CASE

      WHEN ITEMID = 50882 THEN 'BICARBONATE'

      WHEN ITEMID = 50902 THEN  'CHLORIDE'
      WHEN ITEMID = 50806 THEN 'CHLORIDE'

      WHEN ITEMID = 50983  THEN  'SODIUM'
      WHEN ITEMID = 50824  THEN  'SODIUM'

      ELSE null
    END AS label , valuenum
  

FROM icu_18 ie

  LEFT JOIN mimiciii.labevents le
    ON le.subject_id = ie.subject_id AND le.hadm_id = ie.hadm_id
	
    AND le.ITEMID in

    (
      -- comment is: LABEL | CATEGORY | FLUID | NUMBER OF ROWS IN LABEVENTS
     

      50882, -- BICARBONATE | CHEMISTRY | BLOOD | 780733

      50902, -- CHLORIDE | CHEMISTRY | BLOOD | 795568
      50806, -- CHLORIDE, WHOLE BLOOD | BLOOD GAS | BLOOD | 48187

      50983, -- SODIUM | CHEMISTRY | BLOOD | 808489
      50824 -- SODIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 71503

    )
    AND valuenum IS NOT null AND valuenum > 0 -- lab values cannot be 0 and cannot be negative
	where    le.charttime BETWEEN (ie.intime) AND (ie.outtime)) pvt

where day <=1 
and icustay_id in 
(select icustay_id 
from mimiciii.icustays icu join
mimiciii.patients pt on icu.subject_id=pt.subject_id 
where pt.dod > icu.outtime)

"""

df1 = pd.read_sql_query(query, con)

In [ ]:
df1.head()

In [13]:
df1 = df1.filter(['label','valuenum'])

In [15]:
df1.columns = ['vitalid','valuenum']

In [21]:
def iqr(x):
    return x.quantile(0.75) - x.quantile(0.25)

In [36]:
import numpy as np
f = ['mean','median', 'std', iqr]
df2 = df1.groupby('vitalid')['valuenum'].agg(f)
df2

,mean,median,std,iqr
vitalid,,,,
BICARBONATE,25.696443,25.0,5.034271,7.0
CHLORIDE,104.144561,104.0,6.322994,8.0
SODIUM,138.852473,139.0,5.119334,6.0


# Vitals

In [32]:
query = """
select * from 

(select ie.subject_id, ie.hadm_id, ie.icustay_id, ce.charttime,
	ceiling((extract( epoch from ie.outtime- ce.charttime ))/60/60/24) as day 
  , case
    when itemid in (211,220045) and valuenum > 0 and valuenum < 300 then 'HeartRate'
    when itemid in (51,442,455,6701,220179,220050) and valuenum > 0 and valuenum < 400 then 'SysBP'
    when itemid in (8368,8440,8441,8555,220180,220051) and valuenum > 0 and valuenum < 300 then  'DiasBP'

    when itemid in (615,618,220210,224690) and valuenum > 0 and valuenum < 70 then 'RespRate'

    else null end as VitalID
      -- convert F to C
  , case when itemid in (223761,678) then (valuenum-32)/1.8 else valuenum end as valuenum

  from icu_18 ie
  left join mimiciii.chartevents ce
  on ie.subject_id = ce.subject_id and ie.hadm_id = ce.hadm_id and ie.icustay_id = ce.icustay_id
  --and ce.charttime between ie.intime and ie.intime + interval '1' day
  -- exclude rows marked as error
  and ce.error IS DISTINCT FROM 1
  where ce.itemid in
  (
  -- HEART RATE
  211, --"Heart Rate"
  220045, --"Heart Rate"

  -- Systolic/diastolic

  51, --	Arterial BP [Systolic]
  442, --	Manual BP [Systolic]
  455, --	NBP [Systolic]
  6701, --	Arterial BP #2 [Systolic]
  220179, --	Non Invasive Blood Pressure systolic
  220050, --	Arterial Blood Pressure systolic

  8368, --	Arterial BP [Diastolic]
  8440, --	Manual BP [Diastolic]
  8441, --	NBP [Diastolic]
  8555, --	Arterial BP #2 [Diastolic]
  220180, --	Non Invasive Blood Pressure diastolic
  220051, --	Arterial Blood Pressure diastolic


  -- RESPIRATORY RATE
  618,--	Respiratory Rate
  615,--	Resp Rate (Total)
  220210,--	Respiratory Rate
  224690, --	Respiratory Rate (Total)

  -- SPO2, peripheral
  646, 220277

 

  ) and ce.charttime > ie.intime)
  pvt
  
  where day <= 1 and icustay_id in
  (
  select icustay_id 
from mimiciii.icustays icu join
mimiciii.patients pt on icu.subject_id=pt.subject_id 
where pt.dod > icu.outtime
  )

"""

df = pd.read_sql_query(query, con)

In [33]:
df.head()

,subject_id,hadm_id,icustay_id,charttime,day,vitalid,valuenum
0,103,130744,212246,2144-08-15 14:00:00,1.0,SysBP,159.0
1,103,130744,212246,2144-08-15 15:00:00,1.0,SysBP,144.0
2,101,175533,233111,2196-10-05 17:00:00,1.0,SysBP,117.0
3,101,175533,233111,2196-10-05 18:00:00,1.0,SysBP,126.0
4,101,175533,233111,2196-10-05 19:00:00,1.0,SysBP,133.0


In [34]:
df = df.filter(['vitalid','valuenum'])

In [37]:
df3 = df.groupby('vitalid')['valuenum'].agg(f)
df3

,mean,median,std,iqr
vitalid,,,,
DiasBP,59.431493,58.0,14.794056,19.0
HeartRate,82.917012,82.0,16.126807,22.0
RespRate,19.693027,19.0,5.462299,7.0
SysBP,123.279089,121.0,22.821488,31.0


In [38]:
df_final = pd.concat([df2,df3])

In [39]:
df_final

,mean,median,std,iqr
vitalid,,,,
BICARBONATE,25.696443,25.0,5.034271,7.0
CHLORIDE,104.144561,104.0,6.322994,8.0
SODIUM,138.852473,139.0,5.119334,6.0
DiasBP,59.431493,58.0,14.794056,19.0
HeartRate,82.917012,82.0,16.126807,22.0
RespRate,19.693027,19.0,5.462299,7.0
SysBP,123.279089,121.0,22.821488,31.0


In [40]:
df_final.to_csv('standard.csv')

In [41]:
query = '''
with base as
(
  SELECT pvt.ICUSTAY_ID
  , pvt.charttime, pvt.day

  -- Easier names - note we coalesced Metavision and CareVue IDs below
  , max(case when pvt.itemid = 454 then pvt.valuenum else null end) as GCSMotor
  , max(case when pvt.itemid = 723 then pvt.valuenum else null end) as GCSVerbal
  , max(case when pvt.itemid = 184 then pvt.valuenum else null end) as GCSEyes

  -- If verbal was set to 0 in the below select, then this is an intubated patient
  , case
      when max(case when pvt.itemid = 723 then pvt.valuenum else null end) = 0
    then 1
    else 0
    end as EndoTrachFlag

  , ROW_NUMBER ()
          OVER (PARTITION BY pvt.ICUSTAY_ID ORDER BY pvt.charttime ASC) as rn

  FROM  (
  select l.ICUSTAY_ID, ceiling((extract( epoch from b.outtime- l.charttime ))/60/60/24) as day
  -- merge the ITEMIDs so that the pivot applies to both metavision/carevue data
  , case
      when l.ITEMID in (723,223900) then 723
      when l.ITEMID in (454,223901) then 454
      when l.ITEMID in (184,220739) then 184
      else l.ITEMID end
    as ITEMID

  -- convert the data into a number, reserving a value of 0 for ET/Trach
  , case
      -- endotrach/vent is assigned a value of 0, later parsed specially
      when l.ITEMID = 723 and l.VALUE = '1.0 ET/Trach' then 0 -- carevue
      when l.ITEMID = 223900 and l.VALUE = 'No Response-ETT' then 0 -- metavision

      else VALUENUM
      end
    as VALUENUM
  , l.CHARTTIME
  from mimiciii.CHARTEVENTS l

  -- get intime for charttime subselection
  inner join icu_18 b
    on l.icustay_id = b.icustay_id

  -- Isolate the desired GCS variables
  where l.ITEMID in
  (
    -- 198 -- GCS
    -- GCS components, CareVue
    184, 454, 723
    -- GCS components, Metavision
    , 223900, 223901, 220739
  )
  -- Only get data for the first 24 hours
  --and l.charttime between b.intime and b.intime + interval '1' day
  -- exclude rows marked as error
  and l.error IS DISTINCT FROM 1 and l.charttime >= b.intime
  ) pvt
  group by pvt.ICUSTAY_ID, pvt.charttime, day
)
, gcs as (
  select b.*
  , b2.GCSVerbal as GCSVerbalPrev
  , b2.GCSMotor as GCSMotorPrev
  , b2.GCSEyes as GCSEyesPrev
  -- Calculate GCS, factoring in special case when they are intubated and prev vals
  -- note that the coalesce are used to implement the following if:
  --  if current value exists, use it
  --  if previous value exists, use it
  --  otherwise, default to normal
  , case
      -- replace GCS during sedation with 15
      when b.GCSVerbal = 0
        then 15
      when b.GCSVerbal is null and b2.GCSVerbal = 0
        then 15
      -- if previously they were intub, but they aren't now, do not use previous GCS values
      when b2.GCSVerbal = 0
        then
            coalesce(b.GCSMotor,6)
          + coalesce(b.GCSVerbal,5)
          + coalesce(b.GCSEyes,4)
      -- otherwise, add up score normally, imputing previous value if none available at current time
      else
            coalesce(b.GCSMotor,coalesce(b2.GCSMotor,6))
          + coalesce(b.GCSVerbal,coalesce(b2.GCSVerbal,5))
          + coalesce(b.GCSEyes,coalesce(b2.GCSEyes,4))
      end as GCS

  from 
base	b
  -- join to itself within 6 hours to get previous value
  left join base b2
    on b.ICUSTAY_ID = b2.ICUSTAY_ID and b.rn = b2.rn+1 and b2.charttime > b.charttime - interval '6' hour and b2.charttime < b.charttime 
)
, gcs_final as (
  select gcs.*
  -- This sorts the data by GCS, so rn=1 is the the lowest GCS values to keep
  , ROW_NUMBER ()
          OVER (PARTITION BY gcs.ICUSTAY_ID
                ORDER BY gcs.GCS
               ) as IsMinGCS
  from gcs
)
select ie.SUBJECT_ID, ie.HADM_ID, ie.ICUSTAY_ID
-- The minimum GCS is determined by the above row partition, we only join if IsMinGCS=1
, day
, GCS as MinGCS
, coalesce(GCSMotor,GCSMotorPrev) as GCSMotor
, coalesce(GCSVerbal,GCSVerbalPrev) as GCSVerbal
, coalesce(GCSEyes,GCSEyesPrev) as GCSEyes
, EndoTrachFlag as EndoTrachFlag

-- subselect down to the cohort of eligible patients
from mimiciii.icustays ie
inner join gcs_final gs
  on ie.ICUSTAY_ID = gs.ICUSTAY_ID and gs.IsMinGCS = 1
where day<=1 
and ie.ICUSTAY_ID
in
(
select icustay_id 
from mimiciii.icustays icu join
mimiciii.patients pt on icu.subject_id=pt.subject_id 
where pt.dod > icu.outtime
)
ORDER BY ie.ICUSTAY_ID, day
;
'''

df2 = pd.read_sql_query(query, con)

In [42]:
df2['mingcs'].mean()

13.66217630133403

In [43]:
df2['mingcs'].std()

2.3355640788808962

In [44]:
df2['mingcs'].median()

15.0

In [45]:
df2['mingcs'].quantile(0.75) - df2['mingcs'].quantile(0.25)

1.0

In [3]:
df = pd.read_csv('fluid_io.csv')

In [4]:
df.head()

,subject_id,hadm_id,icustay_id,icu_day,fluid_input_ml,fluid_output_ml,fluid_net_input_ml
0,3,145834,211552,2,3736.325974,1200.0,2536.325974
1,3,145834,211552,3,2007.500000,2765.0,-757.500000
2,3,145834,211552,4,1560.000000,5435.0,-3875.000000
3,3,145834,211552,5,980.000000,3320.0,-2340.000000
4,3,145834,211552,6,1364.050000,1960.0,-595.950000


In [6]:
df1 = df.groupby('icustay_id').apply ( lambda x : x.loc[x['icu_day'].argmin(),'fluid_net_input_ml'])
df1.head()

/Users/panyiheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.
  """Entry point for launching an IPython kernel.


icustay_id
200001    1620.813392
200003    3813.181250
200006    -165.000000
200007    1146.999956
200009    -735.000000
dtype: float64

In [7]:
df1 = pd.DataFrame({'icustay_id':df1.index, 'balance':df1.values})
df1.head()

,icustay_id,balance
0,200001,1620.813392
1,200003,3813.181250
2,200006,-165.000000
3,200007,1146.999956
4,200009,-735.000000


In [9]:
df1['balance'].mean(), df1['balance'].median(),df1['balance'].quantile(0.75) - df1['balance'].quantile(0.25), df1['balance'].std()

(907.478118704745, 632.9166655541001, 2334.6600329632, 1835.5565638734388)

In [1]:
import numpy as np
import pandas as pd
import datetime

In [9]:
query = """
select icustay_id 
from mimiciii.icustays icu join
mimiciii.patients pt on icu.subject_id=pt.subject_id 
where pt.dod > icu.outtime

"""

df1 = pd.read_sql_query(query, con)

In [11]:
# Create ICU Id list
icu = []
for icuid in df1['icustay_id']:
    icu.append(icuid)

In [19]:

df = pd.read_csv('chloride.csv')
df[df['icustay_id'].isin(icu)]
#df = df.set_index(['subject_id','hadm_id','icustay_id'])
#df = df.filter(['icustay_id','icu_day','chloride_max','chloride_input_meq'])

,subject_id,hadm_id,icustay_id,icu_day,chloride_input_meq,chloride_max
0,3,145834,211552,1,1861.475000,114.0
1,3,145834,211552,2,241.780000,112.0
2,3,145834,211552,3,124.740000,111.0
3,3,145834,211552,4,73.920000,109.0
4,3,145834,211552,5,73.920000,108.0
5,3,145834,211552,6,70.840000,96.0
21,11,194540,229441,1,35.550000,106.0
22,11,194540,229441,2,49.800000,NaN
23,12,112213,232669,1,159.587500,113.0
24,12,112213,232669,2,162.840000,112.0


In [20]:
df

,subject_id,hadm_id,icustay_id,icu_day,chloride_input_meq,chloride_max
0,3,145834,211552,1,1861.475000,114.0
1,3,145834,211552,2,241.780000,112.0
2,3,145834,211552,3,124.740000,111.0
3,3,145834,211552,4,73.920000,109.0
4,3,145834,211552,5,73.920000,108.0
5,3,145834,211552,6,70.840000,96.0
6,4,185777,294638,1,150.150000,108.0
7,4,185777,294638,2,NaN,106.0
8,6,107064,228232,1,335.720000,101.0
9,6,107064,228232,2,280.280000,102.0


In [22]:
df1 = df.reset_index().groupby('icustay_id').apply ( lambda x :x.loc[x['icu_day'].argmax(),'chloride_input_meq'])

/Users/panyiheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  """Entry point for launching an IPython kernel.


In [24]:
df1 = df1.dropna()

In [34]:
df = pd.DataFrame({'icustay_id':df1.index, 'chloride_input_meq':df1.values})

In [35]:
df['chloride_input_meq'].mean()

109.84520237507292

In [36]:
df['chloride_input_meq'].median()

50.00000052

In [31]:
df1.quantile(0.75) - df1.quantile(0.25)

138.21499384

In [32]:
df1.std()

155.16054625261634

In [33]:
df1

icustay_id
200001     38.500000
200003     92.340000
200007    221.760000
200009     40.000000
200010    464.727083
200014     59.250000
200016    261.222496
200019     38.800000
200021      5.368200
200024     57.903179
200025     20.000000
200028     50.000001
200029      0.392700
200030     15.400000
200031     83.740000
200033    126.601101
200034     23.941867
200035     65.838848
200036    161.700000
200037      0.504000
200038     45.322308
200039     12.320000
200041     61.600000
200044    345.219956
200045    154.000000
200050    211.365000
200052    349.240000
200055     46.200000
200061      6.988111
200062     23.080000
             ...    
299923     36.042553
299925     18.480000
299928     76.999999
299929     22.229900
299930    222.094666
299931     98.175000
299933    240.757575
299941     40.425000
299943     16.478000
299947     13.825201
299948    336.685850
299950     42.132262
299951     25.410000
299954     20.020000
299955      9.240000
299957     13.509733
29